In [2]:
using Flux, Statistics
using Flux: onehotbatch, onecold, crossentropy, throttle, params 
using Base.Iterators: repeated, partition
using Printf, BSON
using CSV
using DataFrames
using Tables

In [11]:
original_dat = CSV.read("../processed-data/otu-yield-per-plant.csv", header=true, DataFrame)
generated_dat = CSV.read("../processed-data/generated-data.csv", header=false, DataFrame)

original = Matrix(original_dat)
generated = Matrix(generated_dat)
data = vcat(original, generated)

otu = data[:,2:60]
label = data[:,62]

percentage = [(i, count(==(i), label)) for i in unique(label)]
println("Number of low-yield samples: ", percentage[1][2], "; Number of high-yield samples: ", percentage[2][2])

Number of low-yield samples: 222; Number of high-yield samples: 222
